<a href="https://colab.research.google.com/github/miguelamda/DL/blob/master/5.%20Redes%20Convolucionales/Practica5.1.%20Introducci%C3%B3n%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRÁCTICA 5.1. INTRODUCCIÓN A LAS REDES CONVOLUCIONALES

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms.v2 as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

Vamos a comenzar mostrando una **Red Convolucional** muy simple para abordar el problema de clasificación MNIST que ya analizamos anteriormente. Con lo que hemos visto hasta ahora no resultará tan extraño, y más adelante detallaremos cada una de las capas que lo componen describiendo la funcionalidad que juegan en la red global. Veremos que, aunque la red convolucional que construiremos de forma directa es muy simple, supera el rendimiento de la red clásica que creamos en el notebook de la práctica 3.2.

Si quieres leer una introducción (no matemática) de cómo y porqué funcionan las redes convolucionales puedes mirar [este magnífico blog de Ujjwal Karn](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets).

![](https://github.com/miguelamda/DL/blob/master/5.%20Redes%20Convolucionales/imgs/CNN.png?raw=1)

## 1. Construyendo una CNN

Como se puede observar en el siguiente código, la red convolucional que vamos a usar está formada esencialmente por dos capas convoluciones bidimensionales (`Conv2d`) seguidas de dos capas max_pooling (`MaxPool2d`). 

A diferencia de `nn.Linear`, que opera sobre vectores aplanados, `nn.Conv2d` opera sobre tensores 2D (imágenes), preservando su estructura espacial. Por tanto, el valor que recibe en sus argumentos cambia, siendo `nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,padding_mode)`:
* `in_channels`: canales de entrada. Por ejemplo, en imágenes en escala de grises sería 1, en imágenes a color RGB serían 3.
* `out_channels`: canales de salida. Corresponde al número de mapas de activación de salida, es decir, el número de neuronas en la capa convolucional (cada neurona aplica la convolución con su kernel y genera un mapa de activación de salida).
* `kernel_size`: tamaño del filtro convolucional (o kernel). Puede ser un entero $n$, dando lugar a un kernel $nxn$, o una tupla que indique otra forma $(m,n)$
* `stride`: desplazamiento o stride aplicado en la convolución. Por defecto, 1.
* `padding` y `padding_mode`: número de píxeles a extender la imagen de entrada mediante padding, según el método padding_mode (por defecto, con ceros).
* Ver más parámetros [en la documentación](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

Para `nn.MaxPool2d` tan solo debemos indicar el tamaño del kernel (igual que en `Conv2d`) y el stride.

**Ejercicio:** Dada la definición de red convolucional siguiente, haz una lista de número de neuronas y tamaño de kernel en cada capa, en la celda siguiente:


In [42]:
CNN = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), 
        nn.Conv2d(32, 64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
)

Escribe aquí tu solución:
* capa 1
* capa 2

Solución:
* Capa 1: convolución de 3x3, de 32 neuronas
* Capa 2: convolución de 3x3, de 64 neuronas


Ahora veamos qué forma tiene esta red convolucional:

In [33]:
from torchsummary import summary
print(CNN)
summary(CNN,verbose=0)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            320
├─ReLU: 1-2                              --
├─MaxPool2d: 1-3                         --
├─Conv2d: 1-4                            18,496
├─ReLU: 1-5                              --
├─MaxPool2d: 1-6                         --
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0

Podemos ver las capas que hemos introducido, y el número de parámetros en cada una. Pero no podemos ver información de la forma que tienen los tensores que genera cada capa. Y es que la convolución en sí no necesita saber la forma del tensor de entrada, ya que aplicará la convolución a toda la imagen de entrada independientemente de su tamaño. Eso sí, el tamaño del tensor de salida (mapa de activación) dependerá del tamaño de entrada. 

Lo que hemos definido ha sido solo la parte convolucional. Para poder hacer clasificación multiclase (para MNIST), debemos usar una o varias capas donde hacer combinación lineal y finalmente aplicar *softmax* para inferir la clase final. 

**Ejercicio**: Como en el clasificador necesitaremos incluir una primera capa lineal, ¿cuál es el número de características que tendría de entrada? Recuerda que para definir dicha capa, `nn.Linear(in_features,out_features`, ¿cuánto es `in_features`? Puedes usar la siguiente fórmula para calcular el tamaño de salida $(W_{out},H_{out})$ de una operación convolucional aplicada a una entrada $(W_{in},H_{in})$ con un kernel de tamaño $(K_w,K_h)$, padding $P$ y stride $S$:

$W_{out} = \left\lfloor \frac{W_{in} - K_w + 2P}{S} \right\rfloor + 1$       

$H_{out} = \left\lfloor \frac{H_{in} - K_h + 2P}{S} \right\rfloor + 1$

Por simplicidad puedes asumir que la imagen de entrada es cuadrada y los kernels también. Es decir, la altura es igual a la anchura, por lo que solo hay que calcular $W$, ya que $H$ es igual.

*Tu solución:*

* Salida primera capa convolucional: 
* Salida primera capa MaxPool: 
* Salida segunda capa convolucional: 
* Salida segunda capa MaxPool: 
* Número de características total al final:

Solución:

Podemos simplificar el cálculo porque las imágenes de entrada son cuadradas ($W_{in}$ = $H_{in}$) y los kernels son cuadrados ($K_w=K_h$, concretamente, son de 3x3).:
* Salida primera capa convolucional: W_out = (28-3+2*0)/1+1 = 26
* Salida primera capa MaxPool: 26/2 = 13
* Salida segunda capa convolucional: W_out = (13-3+2*0)/1+1 = 11
* Salida segunda capa MaxPool: 21/2 = 5
* Número de características total al final: $5*5*64 = 1600$

Aunque normalmente estos cálculos se hacen mano según la fórmula descrita, la función `summary` puede servirnos de ayuda para automatizar este proceso. Le podemos pasar como argumento una tupla que defina el shape de entrada a la red. Los otros dos argumentos, verbose y device, los discutiremos más adelante.

In [34]:
summary(CNN, (1,28,28), verbose=0, device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 26, 26]          320
├─ReLU: 1-2                              [-1, 32, 26, 26]          --
├─MaxPool2d: 1-3                         [-1, 32, 13, 13]          --
├─Conv2d: 1-4                            [-1, 64, 11, 11]          18,496
├─ReLU: 1-5                              [-1, 64, 11, 11]          --
├─MaxPool2d: 1-6                         [-1, 64, 5, 5]            --
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
Total mult-adds (M): 2.42
Input size (MB): 0.00
Forward/backward pass size (MB): 0.22
Params size (MB): 0.07
Estimated Total Size (MB): 0.30

En PyTorch, una red convolucional toma como dato de entrada un tensor de la forma `(image_channels, image_height, image_width)`. En este caso, para ajustarse a las características de las imágenes de MNIST, será `(1, 28, 28)`, ya que usaremos un solo canal (gris) en las imágenes.

Cada capa de tipo `Conv2d()` y `MaxPool2d()` dan como salida un tensor 3D de forma `(channels, height, width)`. Tanto la anchura como altura del tensor tienden a disminuir a medida que avanzamos en la red. El -1 que ves al principio corresponde a la dimensión del batch. Y es que las capas están diseñadas para recibir batches como entrada, no datos únicos.

A continuación, hemos de pasar la salida de la última capa anterior (de tamaño `(64, 5, 5)`) a una red lineal clasificadora similar a las que ya hemos visto en ejemplos anteriores. Como estas capas procesan vectores, que son 1D, y la entrada es un tensor 3D, hemos de aplanar el tensor por medio de la capa `Flatten()`, que también proporciona PyTorch:

In [43]:
clasificador = nn.Sequential(
    nn.Flatten(),
    nn.Linear(64 * 5 * 5, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

Finalmente podemos juntar la parte convolucional con la parte del clasificador en un único modelo. Podemos usar de nuevo `nn.Sequential` como hasta ahora. Esta vez vamos a usar de una forma distinta, mediante un diccionario ordenado, de esta forma le podemos dar un nombre a cada bloque.

In [44]:
from collections import OrderedDict

model = nn.Sequential(
    OrderedDict([
        ("Bloque convolucional",CNN),
        ("Clasificador",clasificador)
    ]))

print(model)
summary(model, (1,28,28), verbose=0, device='cpu')

Sequential(
  (Bloque convolucional): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (Clasificador): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1600, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 5, 5]            --
|    └─Conv2d: 2-1                       [-1, 32, 26, 26]          320
|    └─ReLU: 2-2                         [-1, 32, 26, 26]          --
|    └─MaxPool2d: 2-3                    [-1, 32, 13, 13]          --
|    └─Conv2d: 2-4                       [-1, 64, 11, 11]          18,496
|    └─ReLU: 2-5                         [-1, 64, 11, 11]          --
|    └─MaxPool2d: 2-6                    [-1, 64, 5, 5]            --
├─Sequential: 1-2                        [-1, 10]                  --
|    └─Flatten: 2-7                      [-1, 1600]                --
|    └─Linear: 2-8                       [-1, 128]                 204,928
|    └─ReLU: 2-9                         [-1, 128]                 --
|    └─Linear: 2-10                      [-1, 10]                  1,290
Total params: 225,034
Trainable params: 225,034
Non-trainable params: 0


Como el objetivo es calcular una clasificación en 10 clases, la última capa es una capa densa con 10 unidades. Como vimos en la práctica 3.2, no añadimos `softmax` ya que en PyTorch no es necesario al estar incluido dentro de la función de pérdida.

## 2. Carga de datos y entrenamiento

Una vez definida la red, realizamos el entrenamiento de forma similar a como hicimos en el modelo simple de MNIST:

In [ ]:
transform = transforms.Compose([transforms.PILToTensor(), transforms.ToDtype(torch.float32, scale=True)])

# Descargar y cargar los datos de entrenamiento. Podemos pasarle el transformador a la hora de cargar el dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Descargar y cargar los datos de prueba
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:01<00:00, 7411338.42it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 294739.07it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 2773887.11it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 1031821.96it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Y preparamos el resto de elementos para entrenar al modelo:

In [30]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)  
    return (preds == yb).float().mean().item()

In [45]:
%%time 

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    model.train() # Poner el modelo en modo de entrenamiento
    loss_acum = 0
    for x, y in train_loader:        
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)
        
        # Backward y optimizar
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_acum += loss.item()
    
    train_loss = loss_acum / len(train_loader)
    
    # Evaluar el modelo en datos de prueba para obtener la precisión
    model.eval() # Poner el modelo en modo de evaluación        
    accu_acum = 0
    with torch.no_grad():
        for x, y in test_loader:
            outputs = model(x)
            accu_acum += accuracy(outputs,y)            

    accu = accu_acum / len(test_loader)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Accuracy: {accu:.2f}')

Epoch [1/5], Train Loss: 0.1714, Val Accuracy: 0.98
Epoch [2/5], Train Loss: 0.0505, Val Accuracy: 0.99
Epoch [3/5], Train Loss: 0.0352, Val Accuracy: 0.99
Epoch [4/5], Train Loss: 0.0254, Val Accuracy: 0.99
Epoch [5/5], Train Loss: 0.0200, Val Accuracy: 0.99
CPU times: user 9min 15s, sys: 5.68 s, total: 9min 20s
Wall time: 1min 13s


**Ejercicio opcional**: modifica el código anterior para reportar también la pérdida sobre el conjunto de validación.

Podemos ver que alcanzamos un accuracy del 99%, superando al modelo básico que vimos en la práctica 3.2. Sin embargo, el entrenamiento ha tardado algo más de tiempo, ¿podemos acelerarlo?

## 3. Aceleración con GPUs

Según tu CPU, el bucle de entrenamiento anterior pudo haber tardado más o menos, pero seguro que ha durado uno o más minutos. Y es que, una CPU no es el dispositivo más eficiente para ser usado en el entrenamiento de redes neuronales. La CPU está diseñada para realizar cómputo con instrucciones muy complejas, pero con un nivel de paralelismo limitado (¿cuántos núcleos tiene tu CPU?). En cambio, una **tarjeta gráfica (GPU)** sí que provee la potencia computacional para acelerar los cálculos requeridos. Estamos hablando, principalmente, de **multiplicaciones de matrices** (como la que hicimos en la práctica anterior), y esta operación se puede *paralelizar* muy bien (piensa en calcular, en paralelo, el valor de cada elemento de salida). Una GPU provee de miles de núcleos que pueden, fácilmente, repartirse la carga de trabajo a través de su memoria. Si tienes una GPU de *NVIDIA*, podrás hacer estos cálculos con **CUDA**. Si tienes una GPU de *AMD*, con **ROCm**. Hay otros dispositivos, como las *TPUs* de Google, pero no las usaremos en esta asignatura. Si tienes acceso a una TPU, puedes configurarla también con PyTorch, simplemente busca la ayuda en el manual.

Esta parte de la práctica podrás ejecutarla sin problema una vez estés en un entorno con una GPU (bien sea en local o en la nube). En la siguiente celda ejecutaremos la instrucción que nos muestra las GPUs disponibles en el sistema: `nvidia-smi`. Éste es un programa que se instala junto al driver de CUDA, la plataforma para cálculo paralelo de NVIDIA. La exclamación al comienzo indica que el código no es Python, sino una instrucción a ejecutar en el sistema (p.ej. en la terminal).

In [46]:
!nvidia-smi

Fri Oct 17 16:20:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080         Off| 00000000:01:00.0 Off |                  N/A |
|  0%   47C    P8                3W / 265W|   1044MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

¿Cómo podemos usar una GPU? Fácil, creando una variable indicando `"cuda"` en `torch.device`. Esta variable la usaremos para indicar dónde queremos ejecutar las operaciones. Por seguridad, en caso de no tener GPU, podemos asignar `"cpu"`, y el código siguiente seguiría siendo válido, aunque más lento.

Es posible que estemos en un servidor con varias GPUs, en tal caso podemos elegir solo una GPU. Para ello, hay dos formas:
* Indicando `"cuda:X"` en `torch.device`, con X el id del dispositivo (por defecto, X es 0). Es decir, si quieres elegir la segunda GPU, indica `"cuda:1"`.
* Definiendo la variable entorno `CUDA_VISIBLE_DEVICES` con el id de la GPU a usar. Por ejemplo, para usar solo la segunda GPU, ejecutar: 
```bash
export CUDA_VISIBLE_DEVICES="1"
```

Por ahora usaremos solo la GPU por defecto.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # por defecto GPU 0
#device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # elegir la GPU 1, si la hay
torch.cuda.is_available()

True

Lo primero a tener en cuenta a la hora de usar una GPU, es que éste dispositivo solo tiene acceso a su propia memoria. Esto quiere decir que todo dato que queremos que toque la GPU, se lo tenemos que enviar. En concreto, demos enviar a la GPU tanto el modelo (sus parámetros deben estar allí para ser operados) como cada batch de datos. Debemos tener en cuenta una serie de restricciones:
* Las GPUs tienen una **memoria limitada**, que va desde los 8GB a los 80GB. Si usamos una GPU de gama baja, es posible que tengamos 8GB, esto significa que debemos llevar cuidado con el tamaño de batch que le enviemos. Por eso es importante trabajar con batch de datos, en vez de con el dataset al completo, ya que éste no suele caber al completo en una GPU.
* Este flujo de datos hacia y desde la GPU suele ser el mayor **cuello de botella**, por lo que hay que limitarlo al máximo.

A continuación verás cómo podemos enviar datos a la GPU. Los tensores cambiarán de estado y pasarán a ser tensores residentes en la GPU. Hay que tener precaución al usarlos, ya que siempre que queramos operar con ellos, necesitaremos hacerlo con otros tensores que también estén en la GPU. Si pedimos operar con un tensor residente en la CPU y un tensor residente en la GPU, la operación fallará.

In [49]:
x0 = train_dataset[0][0] # La X del primer ejemplo
print(x0.device)  # este tensor está en la CPU

x0_gpu = x0.cuda() # lo copiamos a la GPU
x0_gpu.device   # este nuevo tensor está en la GPU


cpu


device(type='cuda', index=0)

Hay otra forma de copiar datos a la GPU, más robusta y flexible, es la que verás abajo. Se basa en la variable device, de esta forma:
* Si no tienes GPU, device será la CPU, por lo que el código será redundante pero no fallará.
* Esta función permite copiar los datos a una GPU que no sea la 0.

In [52]:
x0_gpu2 = x0.to(device)
print(x0_gpu2.device)
# como no vamos a usar esta variable, es una buena práctica
# eliminarla para liberar memoria 
del(x0_gpu2) 

cuda:0


Vamos a crear de nuevo un modelo, con la configuración anterior. 

**Ejercicio:** Cambia el siguente FIXME con la misma secuencia de capas del modelo anterior. Esta vez, por simplicidad, puedes definir todas las capas juntas, sin separar por bloque convolucional y clasificador.

In [ ]:
model_gpu = nn.Sequential( FIXME )
model_gpu

In [63]:
# Solución
model_gpu = nn.Sequential(        
    nn.Conv2d(1, 32, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2), 
    nn.Conv2d(32, 64, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(64 * 5 * 5, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
    )
model_gpu

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=1600, out_features=128, bias=True)
  (8): ReLU()
  (9): Linear(in_features=128, out_features=10, bias=True)
)

Ahora, podemos copiar el modelo a la GPU. La operación se hace "in-place" para los modelos, es decir, todo el modelo pasa a estar residente en la GPU. Por último, creamos el optimizador en la GPU, que se hará así porque los parámetros están allí.

In [64]:
model_gpu.to(device)  # copiamos el modelo al dispositivo (GPU) elegido

criterion = nn.CrossEntropyLoss()
optimizer_gpu = optim.Adam(model_gpu.parameters(), lr=0.001) # asignamos los parámetros al optimizador

Y por último, redefinimos los bucles de entrenamiento y validación. En este caso vamos a cambiarle el nombre para no confundirlos con la versión para CPU, así podemos usar el modelo nuevo en la GPU, así como el optimizador correspondiente. 

**Ejercicio**: En la línea donde copiamos los datos a la GPU, ¿por qué hay que copiar también `y`?

In [65]:
%%time

num_epochs = 5

for epoch in range(num_epochs):
    model_gpu.train() # Poner el modelo en modo de entrenamiento
    loss_acum = 0
    for x, y in train_loader: 
        # Copiar datos a la GPU
        x = x.to(device)
        y = y.to(device)       

        # Forward pass
        outputs = model_gpu(x)
        loss = criterion(outputs, y)
        
        # Backward y optimizar
        optimizer_gpu.zero_grad()
        loss.backward()
        optimizer_gpu.step()
        
        loss_acum += loss.item()
    
    train_loss = loss_acum / len(train_loader)
    
    # Evaluar el modelo en datos de prueba para obtener la precisión
    model_gpu.eval() # Poner el modelo en modo de evaluación        
    accu_acum = 0
    with torch.no_grad():
        for x, y in test_loader:
            # Copiar datos a la GPU
            x = x.to(device)
            y = y.to(device)       

            outputs = model_gpu(x)
            accu_acum += accuracy(outputs,y)            

    accu = accu_acum / len(test_loader)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Accuracy: {accu:.2f}')

Epoch [1/5], Train Loss: 0.1916, Val Accuracy: 0.98
Epoch [2/5], Train Loss: 0.0558, Val Accuracy: 0.98
Epoch [3/5], Train Loss: 0.0379, Val Accuracy: 0.99
Epoch [4/5], Train Loss: 0.0272, Val Accuracy: 0.99
Epoch [5/5], Train Loss: 0.0216, Val Accuracy: 0.99
CPU times: user 38.9 s, sys: 88.1 ms, total: 39 s
Wall time: 38.4 s


*Solución:* la función de pérdida necesita comparar `y` con las predicciones del modelo `outputs`. Como el modelo está en la GPU, `outputs` es un tensor en la GPU. Por tanto, esta comparación requiere que, o bien `y` y `outputs` estén en el mismo dispositivo. Como la GPU suele ser más rápido en cálculo tensorial, podeos aprovechar que `outputs` está allí, y tan solo copiamos `y`. Además, `y` es un tensor más pequeño y su transferencia es más rápida (recuerda, es un vector de enteros). Alternativamente, podríamos haber copiado `outputs` a la CPU, pero habría sido algo más lento.

¿Cuánto de rápido ha ido en comparación con la CPU? Esto se suele medir como la aceleración (sin unidades, tan solo poniendo una *x* al final), dividiendo el tiempo más lento entre el más rápido. En este caso, tiempo en CPU / tiempo en GPU. ¿Qué aceleración obtienes? En mi caso, como verás, en CPU tardó 1 minuto y 13,9 segundos, y en la GPU 38,4 segundos, esto hace aproximadamente 73,9/38,4 = 1,92x más rápido. Aunque es una pequeña aceleración (92% más rápido), esto es debido a que el tamaño de la red es pequeña y no merece mucho la pena utilizar la GPU. Recuerda que usar la GPU conyeva una también copiar datos a la GPU, lo cual suele ser el principal cuello de botella. Cuanto mayor sea el modelo, mayor será el impacto la GPU.

Desde la versión 2 de PyTorch, es posible optimizar el modelo para que su manejo sea más eficiente. Esto se hace de forma automática, y se consigue con [`compile`](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html).

*Atención: Si estás trabajando con un entorno local, y obtienes errores más adelante a la hora de usar el modelo, puede ser que sea por compilar el modelo. Y es que la compilación genera código C++, por lo que debes tener instalado en local el paquete build-essential y python-dev*.

In [ ]:
#model = torch.compile(model)
#model

## 2. Ejercicio

Prueba a repetir el proceso con otro conjunto de datos similar, como es [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), un dataset que pretende reemplazar MNIST como datos para analizar algoritmos de Machine Learning.

![fashion-mnist](https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png)

## 3. Conclusiones

* Con redes convolucionales, podemos emplear capas bidimensionales para poder tratar con imágenes de forma natural, sin tener que aplanarlas, donde se pierde información.
* Requiere usar una capa flattern (aplanado) para poder pasar de la parte convolucional a la fully connected.